# 卷积神经⽹络（LeNet）
在[“多层感知机的从零开始实现”](../3dlbasic/3.9mlp-scratch.ipynb) ⼀节⾥我们构造了⼀个含单隐藏层的多层感知机模型来
对Fashion-MNIST数据集中的图像进⾏分类。每张图像⾼和宽均是28像素。我们将图像中的像
素逐⾏展开，得到⻓度为784的向量，并输⼊进全连接层中。然而，这种分类⽅法有⼀定的局限
性。
1. 图像在同⼀列邻近的像素在这个向量中可能相距较远。它们构成的模式可能难以被模型识别。
2. 对于⼤尺⼨的输⼊图像，使⽤全连接层容易导致模型过⼤。假设输⼊是⾼和宽均为1000像
素的彩⾊照⽚（含3个通道）。即使全连接层输出个数仍是256，该层权重参数的形状也
是3,000,000 × 256：它占⽤了⼤约3GB的内存或显存。这会带来过于复杂的模型和过⾼的存储开销。

卷积层尝试解决这两个问题。⼀⽅⾯，卷积层保留输⼊形状，使图像的像素在⾼和宽两个⽅向上
的相关性均可能被有效识别；另⼀⽅⾯，卷积层通过滑动窗口将同⼀卷积核与不同位置的输⼊重
复计算，从而避免参数尺⼨过⼤。

卷积神经⽹络就是含卷积层的⽹络。本节⾥我们将介绍⼀个早期⽤来识别⼿写数字图像的卷积神
经⽹络： LeNet [1]。这个名字来源于LeNet论⽂的第⼀作者Yann LeCun。 LeNet展⽰了通过梯度
下降训练卷积神经⽹络可以达到⼿写数字识别在当时最先进的结果。这个奠基性的⼯作第⼀次将
卷积神经⽹络推上舞台，为世⼈所知。
## LeNet模型
LeNet分为卷积层块和全连接层块两个部分。下⾯我们分别介绍这两个模块

卷积层块⾥的基本单位是卷积层后接最⼤池化层：卷积层⽤来识别图像⾥的空间模式，如线条和
物体局部，之后的最⼤池化层则⽤来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单
位重复堆叠构成。在卷积层块中，每个卷积层都使⽤5 × 5的窗口，并在输出上使⽤sigmoid激活
函数。第⼀个卷积层输出通道数为6，第⼆个卷积层输出通道数则增加到16。这是因为第⼆个卷
积层⽐第⼀个卷积层的输⼊的⾼和宽要小，所以增加输出通道使两个卷积层的参数尺⼨类似。卷
积层块的两个最⼤池化层的窗口形状均为2 × 2，且步幅为2。由于池化窗口与步幅形状相同，池
化窗口在输⼊上每次滑动所覆盖的区域互不重叠。

卷积层块的输出形状为(批量⼤小, 通道, ⾼, 宽)。当卷积层块的输出传⼊全连接层块时，全连接
层块会将小批量中每个样本变平（flatten）。也就是说，全连接层的输⼊形状将变成⼆维，其中第
⼀维是小批量中的样本，第⼆维是每个样本变平后的向量表⽰，且向量⻓度为通道、⾼和宽的乘
积。全连接层块含3个全连接层。它们的输出个数分别是120、 84和10，其中10为输出的类别个数。

下⾯我们通过Sequential类来实现LeNet模型

In [2]:
import d2lzh as d2l
import mxnet as mx
from mxnet import nd,init,gluon,autograd
from mxnet.gluon import loss as gloss, nn
import time

net = nn.Sequential()
net.add(nn.Conv2D(channels=6, kernel_size=5, activation='sigmoid'),
        nn.MaxPool2D(pool_size=2,strides=2),
        nn.Conv2D(channels=16,kernel_size=5,activation='sigmoid'),
        nn.MaxPool2D(pool_size=2,strides=2),
        # Dense会默认将(批量⼤⼩, 通道, ⾼, 宽)形状的输⼊转换成
        # (批量⼤⼩, 通道 * ⾼ * 宽)形状的输⼊
        nn.Dense(120, activation='sigmoid'),
        nn.Dense(84, activation='sigmoid'),
        nn.Dense(10))

接下来我们构造⼀个⾼和宽均为28的单通道数据样本，并逐层进⾏前向计算来查看每个层的输出形状。

In [3]:
X = nd.random.uniform(shape=(1,1,28,28))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv0 output shape:	 (1, 6, 24, 24)
pool0 output shape:	 (1, 6, 12, 12)
conv1 output shape:	 (1, 16, 8, 8)
pool1 output shape:	 (1, 16, 4, 4)
dense0 output shape:	 (1, 120)
dense1 output shape:	 (1, 84)
dense2 output shape:	 (1, 10)


可以看到，在卷积层块中输⼊的⾼和宽在逐层减小。卷积层由于使⽤⾼和宽均为5的卷积核，从
而将⾼和宽分别减小4，而池化层则将⾼和宽减半，但通道数则从1增加到16。全连接层则逐层减
少输出个数，直到变成图像的类别数10。    
## 获取数据和训练模型
下⾯我们来实验LeNet模型。实验中，我们仍然使⽤Fashion-MNIST作为训练数据集。

In [4]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

因为卷积神经⽹络计算⽐多层感知机要复杂，建议使⽤GPU来加速计算。我们尝试在gpu(0)上
创建NDArray，如果成功则使⽤gpu(0)，否则仍然使⽤CPU

In [5]:
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,),ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx
ctx = try_gpu()
ctx

cpu(0)

相应地，我们对[“softmax回归的从零开始实现”](../3dlbasic/3.6softmax-regression-scratch.ipynb)⼀节中描述的`evaluate_accuracy`函数略作
修改。由于数据刚开始存在CPU使⽤的内存上，当ctx变量代表GPU及相应的显存时，我们通过
[“GPU计算”](../4compute/4.6use-gpu.ipynb) ⼀节中介绍的`as_in_context`函数将数据复制到显存上，例如gpu(0)。

In [6]:
def evaluate_accuracy(data_iter, net, ctx):
	acc_sum, n = nd.array([0], ctx=ctx), 0
	for X, y in data_iter:
		# 如果ctx代表GPU及相应的显存，将数据复制到显存上
		X, y = X.as_in_context(ctx), y.as_in_context(ctx).astype('float32')
		acc_sum += (net(X).argmax(axis=1) == y).sum()
		n += y.size
	return acc_sum.asscalar() / n        

我们同样对[“softmax回归的从零开始实现”](../3dlbasic/3.6softmax-regression-scratch.ipynb) ⼀节中定义的`train_ch3`函数略作修改，确保计算
使⽤的数据和模型同在内存或显存上。

In [7]:
def train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,
			  num_epochs):
	print('training on', ctx)
	loss = gloss.SoftmaxCrossEntropyLoss()
	for epoch in range(num_epochs):
		train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
		for X, y in train_iter:
			X, y = X.as_in_context(ctx), y.as_in_context(ctx)
			with autograd.record():
				y_hat = net(X)
				l = loss(y_hat, y).sum()
			l.backward()
			trainer.step(batch_size)
			y = y.astype('float32')
			train_l_sum += l.asscalar()
			train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
			n += y.size
		test_acc = evaluate_accuracy(test_iter, net, ctx)
		print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
			  'time %.1f sec'
			  % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
				 time.time() - start))

我们重新将模型参数初始化到设备变量ctx之上，并使⽤Xavier随机初始化。损失函数和训练算
法则依然使⽤交叉熵损失函数和小批量随机梯度下降。

In [8]:
lr, num_epochs = 0.9, 5
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on cpu(0)
epoch 1, loss 2.3182, train acc 0.103, test acc 0.100, time 38.2 sec
epoch 2, loss 1.9270, train acc 0.258, test acc 0.573, time 32.9 sec
epoch 3, loss 0.9514, train acc 0.627, test acc 0.716, time 31.7 sec
epoch 4, loss 0.7516, train acc 0.708, test acc 0.733, time 34.0 sec
epoch 5, loss 0.6668, train acc 0.736, test acc 0.746, time 33.4 sec


## ⼩结
- 卷积神经⽹络就是含卷积层的⽹络。
- LeNet交替使⽤卷积层和最⼤池化层后接全连接层来进⾏图像分类。